<a href="https://colab.research.google.com/github/damunosor/damunosor.github.io/blob/main/GEE_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import ee
from IPython.display import Image
import geemap
import matplotlib.pyplot as plt
import pandas as pd
import json


Get Credentials

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=E4zho84LEAXWJFINYju6vKO9GMOIqchReXJAtgH5lSA&tc=d8-CjTkoSWdTezIZ09YaHC9bKRIPJ8_WWerl6OorfOg&cc=R_wjjkOjczUIjtskYKOVokH1-luUUCIwdOHTsnNnhvg

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXmhGU3zGRVz3fHhqoTrqXb4GMjSJGGdcebnjufMytap6RgVS66j2UA

Successfully saved authorization token.


In [ ]:
Area= ee.Geometry({
  'type': 'Polygon',
  'coordinates':
    [[[-3.1740628610911026,50.72825823496381],
      [-2.5890408884348526,50.72825823496381],
      [-2.5890408884348526,50.91045405059609],
      [-3.1740628610911026,50.72825823496381]]]
})


In [ ]:
# Load Sentinel-1 ImageCollection
vvVhIw= (ee.ImageCollection('COPERNICUS/S1_GRD')
                    .filterBounds(Area)
                    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                    .filter(ee.Filter.eq('instrumentMode', 'IW'))
                    .select('VH'))


In [ ]:
# Separate ascending and descending orbit images into distinct collections.
vvVhIwAsc = vvVhIw.filter(
  ee.Filter.eq('orbitProperties_pass', 'ASCENDING'));
vvVhIwDesc = vvVhIw.filter(
  ee.Filter.eq('orbitProperties_pass', 'DESCENDING'));

In [ ]:
# Mean VH for combined ascending and descending image collections.
vhIwAscDescMean = vvVhIwAsc.merge(vvVhIwDesc).select('VH');

In [ ]:
# Summer (June-August)-2017
Summer_17=vhIwAscDescMean.filterDate('2017-06-01', '2017-09-01').median().clip(Area)
print(Summer_22)
#Summer (June-August)-2023
Summer_23=vhIwAscDescMean.filterDate('2023-06-01', '2023-09-01').median().clip(Area)
print(Summer_23)

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.clip",
    "arguments": {
      "geometry": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Polygon",
          "arguments": {
            "coordinates": {
              "constantValue": [
                [
                  [
                    -76.6823424952994,
                    3.264417133243828
                  ],
                  [
                    -76.52716061053377,
                    3.264417133243828
                  ],
                  [
                    -76.52716061053377,
                    3.3692978939924347
                  ],
                  [
                    -76.6823424952994,
                    3.264417133243828
                  ]
                ]
              ]
            }
          }
        }
      },
      "input": {
        "functionInvocationValue": {
          "functionName": "reduce.median",
          "arguments": {
          

In [ ]:
#Trees
if Summer_17.lt(-10) and Summer_17.gt(-18):
                    Trees_17 = Summer_17.lt(-10) and Summer_17.gt(-18)
if Summer_23.lt(-10) and Summer_23.gt(-18):
                    Trees_23 = Summer_23.lt(-10) and Summer_23.gt(-18)

Trees_17_filter = Trees_17.updateMask(Trees_17).multiply(Trees_17)
Trees_23_filter = Trees_23.updateMask(Trees_23).multiply(Trees_23)



In [ ]:
import geemap

#Set Visualization Parameters
vis_params = { 'min': -18, 'max': -10, 'palette': ['black']}


# Create the map object.
m = geemap.Map(zoom=3)

m.add_ee_layer(Trees_17_filter, vis_params, 'Trees 2017')
# Display the map.
display(m)

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
import folium

# Define the center of our map.
lat, lon = 50.7694044473425,-2.787240897582275

my_map = folium.Map(location=[lat, lon], zoom_start=15)
my_map

def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer




In [ ]:
#Set Visualization Parameters
vis_params = { 'min': -18, 'max': -10, 'palette': ['black']}


# Add the layer to the map object.
my_map.add_ee_layer(Trees_17_filter, vis_params, 'Trees 2017')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)